In [5]:
import pandas as pd
import numpy as np
import wfdb
import ast
import json
import os

In [6]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

path = '/home/lrbutler/Desktop/ptb-xl/'
sampling_rate=100

# Load up the default database

In [7]:
# load and convert annotation data
files = pd.read_csv(os.path.join(path,'ptbxl_database.csv'), index_col='ecg_id')
files.scp_codes = files.scp_codes.apply(lambda x: ast.literal_eval(x))
files[['age', 'sex', 'height', 'weight', 'scp_codes']]

,age,sex,height,weight,scp_codes
ecg_id,,,,,
1,56.0,1,NaN,63.0,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}"
2,19.0,0,NaN,70.0,"{'NORM': 80.0, 'SBRAD': 0.0}"
3,37.0,1,NaN,69.0,"{'NORM': 100.0, 'SR': 0.0}"
4,24.0,0,NaN,82.0,"{'NORM': 100.0, 'SR': 0.0}"
5,19.0,1,NaN,70.0,"{'NORM': 100.0, 'SR': 0.0}"
...,...,...,...,...,...
21833,67.0,1,NaN,NaN,"{'NDT': 100.0, 'PVC': 100.0, 'VCLVH': 0.0, 'ST..."
21834,300.0,0,NaN,NaN,"{'NORM': 100.0, 'ABQRS': 0.0, 'SR': 0.0}"
21835,59.0,1,NaN,NaN,"{'ISCAS': 50.0, 'SR': 0.0}"


In [8]:
print(files.scp_codes)

ecg_id
1                 {'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}
2                             {'NORM': 80.0, 'SBRAD': 0.0}
3                               {'NORM': 100.0, 'SR': 0.0}
4                               {'NORM': 100.0, 'SR': 0.0}
5                               {'NORM': 100.0, 'SR': 0.0}
                               ...                        
21833    {'NDT': 100.0, 'PVC': 100.0, 'VCLVH': 0.0, 'ST...
21834             {'NORM': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
21835                           {'ISCAS': 50.0, 'SR': 0.0}
21836                           {'NORM': 100.0, 'SR': 0.0}
21837                           {'NORM': 100.0, 'SR': 0.0}
Name: scp_codes, Length: 21799, dtype: object


# Check which normal codes are less than 100 percent confidence, remove them

In [9]:
def less_than_100_check(row):
    codes = row['scp_codes']
    if 'NORM' in codes and codes['NORM'] < 100:
        return 1
    else:
        return 0
    
removal_mask = files.apply(less_than_100_check, axis=1).values
filtered_files = files[removal_mask == 0]
print(len(filtered_files))

19457


# Apply high level diagnostic codes for each of the classes

In [25]:
# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
filtered_files['diagnostic_superclass'] = filtered_files.scp_codes.apply(aggregate_diagnostic)
filtered_files['diagnostic_superclass'].head(20)

ecg_id
1     [NORM]
3     [NORM]
4     [NORM]
5     [NORM]
6     [NORM]
7     [NORM]
8       [MI]
9     [NORM]
10    [NORM]
13    [NORM]
14    [NORM]
15    [NORM]
16    [NORM]
19    [NORM]
21    [NORM]
22    [STTC]
24    [NORM]
25    [NORM]
26    [STTC]
28    [STTC]
Name: diagnostic_superclass, dtype: object

# Count how many of each class are present

In [13]:
diagnostic_superclass = filtered_files['diagnostic_superclass'].value_counts()
print(diagnostic_superclass)
classes = diagnostic_superclass.index
counts = diagnostic_superclass.values
frame = pd.DataFrame([co for co in zip(classes, counts)], columns=['class', 'counts'])

diagnostic_superclass
[NORM]                 7004
[MI]                   2532
[STTC]                 2400
[CD]                   1708
[MI, CD]               1297
[HYP, STTC]             781
[MI, STTC]              599
[HYP]                   535
[CD, STTC]              471
[]                      411
[HYP, MI, STTC]         361
[HYP, CD]               300
[MI, CD, STTC]          223
[HYP, CD, STTC]         211
[HYP, MI]               183
[HYP, MI, CD, STTC]     156
[NORM, CD]              143
[HYP, MI, CD]           117
[NORM, STTC]             22
[HYP, NORM]               2
[NORM, STTC, CD]          1
Name: count, dtype: int64


## Remove classes below the count of 20, as well as ones which contain NORM as well as other labels.

In [26]:
# Convert the index to a list and drop rows based on a condition
def convert_and_check(row):
    classes = row['class']
    count = row['counts']
    if 'NORM' in classes and len(classes) > 1:
        return 1    # Drop
    elif len(classes) == 0:
        return 1
    elif count < 20:
        return 1
    else:
        return 0

drop_indices = frame.apply(convert_and_check, axis=1).values
filtered_classes = frame[drop_indices == 0]
print(filtered_classes)

                  class  counts
0                [NORM]    7004
1                  [MI]    2532
2                [STTC]    2400
3                  [CD]    1708
4              [MI, CD]    1297
5           [HYP, STTC]     781
6            [MI, STTC]     599
7                 [HYP]     535
8            [CD, STTC]     471
10      [HYP, MI, STTC]     361
11            [HYP, CD]     300
12       [MI, CD, STTC]     223
13      [HYP, CD, STTC]     211
14            [HYP, MI]     183
15  [HYP, MI, CD, STTC]     156
17        [HYP, MI, CD]     117


## Remove the undesired classes from the dataset

In [15]:
def all_values_match(x, y):
    if len(x) != len(y):
        return 0
    else:
        for value in x:
            if value not in y:
                return 0
    return 1

def in_class_list(row):
    try:
        for allowable_classes in filtered_classes['class'].values:
            if all_values_match(allowable_classes, row['diagnostic_superclass']):
                return 1
        return 0
    
    except:
        print(row['diagnostic_superclass'])
        print(filtered_classes['class'].values)

keep_indices = filtered_files.apply(in_class_list, axis=1)
filtered_files = filtered_files[keep_indices == 1]


In [16]:
len(filtered_files)

18878

# Label 0 for normal, 1 for abnormal

In [17]:
def is_normal(row):
    if 'NORM' in row['diagnostic_superclass']:
        return 1
    else:
        return 0

In [18]:
norm_col = filtered_files.apply(is_normal, axis=1)

filtered_files['NORM'] = norm_col
filtered_files['ABNORM'] = (filtered_files['NORM'] == 0).astype(int)

In [19]:
filtered_files

,patient_id,age,sex,height,weight,nurse,site,device,recording_date,report,...,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr,diagnostic_superclass,NORM,ABNORM
ecg_id,,,,,,,,,,,,,,,,,,,,,
1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,...,NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr,[NORM],1,0
3,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,...,NaN,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr,[NORM],1,0
4,17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,sinusrhythmus normales ekg,...,NaN,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr,[NORM],1,0
5,17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,sinusrhythmus normales ekg,...,NaN,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr,[NORM],1,0
6,19005.0,18.0,1,NaN,58.0,2.0,0.0,CS-12 E,1984-11-28 13:32:13,sinusrhythmus normales ekg,...,NaN,NaN,NaN,NaN,4,records100/00000/00006_lr,records500/00000/00006_hr,[NORM],1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21833,17180.0,67.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-05-31 09:14:35,ventrikulÄre extrasystole(n) sinustachykardie ...,...,NaN,NaN,1ES,NaN,7,records100/21000/21833_lr,records500/21000/21833_hr,[STTC],0,1
21834,20703.0,300.0,0,NaN,NaN,1.0,2.0,AT-60 3,2001-06-05 11:33:39,sinusrhythmus lagetyp normal qrs(t) abnorm ...,...,NaN,NaN,NaN,NaN,4,records100/21000/21834_lr,records500/21000/21834_hr,[NORM],1,0
21835,19311.0,59.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-06-08 10:30:27,sinusrhythmus lagetyp normal t abnorm in anter...,...,NaN,NaN,NaN,NaN,2,records100/21000/21835_lr,records500/21000/21835_hr,[STTC],0,1


In [20]:
len(filtered_files[filtered_files['NORM'] == 0])/len(filtered_files)

0.6289861214111664

In [21]:
filtered_files.to_json(os.path.join(path, 'updated_ptbxl_database.json'))